In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Cài đặt thư viện
!pip install -q accelerate bitsandbytes diffusers transformers ftfy Pillow tensorboard torch torchvision xformers einops

# Tải mã nguồn diffusers (chứa script training)
!git clone https://github.com/huggingface/diffusers
%cd diffusers
!pip install . # Cài đặt diffusers từ mã nguồn vừa tải
%cd examples/dreambooth
!pip install -r requirements.txt # Cài các thư viện phụ thuộc cho script
%cd /content/ # Quay lại thư mục gốc của Colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import os
from PIL import Image
import shutil

# --- 1. ĐỊNH NGHĨA ĐƯỜNG DẪN ---
# Đường dẫn gốc trên Google Drive
drive_base_path = "/content/drive/MyDrive" # Giữ nguyên nếu MyDrive là gốc 

# Đường dẫn tới dữ liệu gốc (128x128)
original_anime_train_dir = os.path.join(drive_base_path, "data/AvatarAnime/AnimeFace/train")

# Đường dẫn tới thư mục CHỨA dữ liệu đã resize (512x512) trên Colab runtime (cho nhanh)
# Tạo thư mục này
resized_data_parent_dir = "/content/anime_data_512"

# Đường dẫn tới thư mục huấn luyện LoRA (chứa thư mục con với ảnh)
lora_training_data_dir = "/content/anime_lora_data"

# Thư mục lưu trữ LoRA weights sau khi train xong
lora_output_dir = os.path.join(drive_base_path, "lora_output/my_anime_style") # Lưu lại vào Drive

# --- 2. TẠO CÁC THƯ MỤC CẦN THIẾT ---
os.makedirs(resized_data_parent_dir, exist_ok=True)
os.makedirs(lora_training_data_dir, exist_ok=True)
os.makedirs(lora_output_dir, exist_ok=True)

# --- 3. RESIZE ẢNH TỪ 128x128 SANG 512x512 ---
target_size = 512
print(f"Bắt đầu resize ảnh từ: {original_anime_train_dir}")
print(f"Lưu ảnh đã resize vào: {resized_data_parent_dir}")

# Đếm số lượng ảnh resize thành công
resized_count = 0
if not os.path.exists(original_anime_train_dir):
    print(f"LỖI: Không tìm thấy thư mục dữ liệu gốc: {original_anime_train_dir}")
else:
    for filename in os.listdir(original_anime_train_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.webp')):
            try:
                img_path = os.path.join(original_anime_train_dir, filename)
                img = Image.open(img_path).convert("RGB") # Mở ảnh và đảm bảo là RGB
                img_resized = img.resize((target_size, target_size), Image.LANCZOS) # Resize

                # Lưu ảnh đã resize vào thư mục tạm trên Colab
                output_path = os.path.join(resized_data_parent_dir, filename)
                img_resized.save(output_path)
                resized_count += 1
            except Exception as e:
                print(f"Lỗi khi xử lý file {filename}: {e}")

    print(f"Hoàn thành resize {resized_count} ảnh sang kích thước {target_size}x{target_size}.")

# --- 4. TẠO CẤU TRÚC THƯ MỤC CHO SCRIPT LORA ---
# Script LoRA cần ảnh nằm trong thư mục con có tên theo format: [TriggerWord]_[ClassDescription]
# Ví dụ: myAnimeStyleToken_anime_face
trigger_word = "myAnimeStyleToken" # Có thể đổi từ này, nhưng phải là từ lạ, duy nhất
class_name = "anime_face"
instance_folder_name = f"{trigger_word}_{class_name}"
instance_data_path = os.path.join(lora_training_data_dir, instance_folder_name)

print(f"Tạo thư mục instance cho LoRA: {instance_data_path}")
os.makedirs(instance_data_path, exist_ok=True)

# --- 5. SAO CHÉP ẢNH ĐÃ RESIZE VÀO THƯ MỤC INSTANCE ---
print(f"Sao chép ảnh từ {resized_data_parent_dir} vào {instance_data_path}")
copied_count = 0
for filename in os.listdir(resized_data_parent_dir):
    source_file = os.path.join(resized_data_parent_dir, filename)
    destination_file = os.path.join(instance_data_path, filename)
    try:
        shutil.copyfile(source_file, destination_file)
        copied_count += 1
    except Exception as e:
        print(f"Lỗi khi sao chép file {filename}: {e}")

print(f"Hoàn thành sao chép {copied_count} ảnh vào thư mục huấn luyện LoRA.")

# Dọn dẹp thư mục chứa ảnh resize tạm thời (không bắt buộc, để tiết kiệm dung lượng Colab)
# print(f"Xóa thư mục ảnh resize tạm: {resized_data_parent_dir}")
# shutil.rmtree(resized_data_parent_dir)

Bắt đầu resize ảnh từ: /content/drive/MyDrive/data/AvatarAnime/AnimeFace/train
Lưu ảnh đã resize vào: /content/anime_data_512
Hoàn thành resize 800 ảnh sang kích thước 512x512.
Tạo thư mục instance cho LoRA: /content/anime_lora_data/myAnimeStyleToken_anime_face
Sao chép ảnh từ /content/anime_data_512 vào /content/anime_lora_data/myAnimeStyleToken_anime_face
Hoàn thành sao chép 800 ảnh vào thư mục huấn luyện LoRA.


In [6]:
%%bash
# Thêm %%bash vào dòng đầu tiên để chạy toàn bộ cell như một shell script

# --- ĐỊNH NGHĨA BIẾN MÔI TRƯỜNG CHO SCRIPT ---
export MODEL_NAME="runwayml/stable-diffusion-v1-5" # Base model
export INSTANCE_DIR="/content/anime_lora_data" # Thư mục chứa thư mục [trigger]_[class] với ảnh 512x512
export OUTPUT_DIR="/content/drive/MyDrive/lora_output/my_anime_style" # Nơi lưu LoRA weights trên Drive
export CLASS_NAME="anime_face" # Tên class (phần sau trigger word trong tên thư mục)
export TRIGGER_WORD="myAnimeStyleToken" # Từ kích hoạt

# --- CHẠY SCRIPT HUẤN LUYỆN ---
# Lệnh accelerate launch vẫn giữ nguyên, CHỈ XÓA dòng --max_train_steps
accelerate launch /content/diffusers/examples/dreambooth/train_dreambooth_lora.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="$TRIGGER_WORD $CLASS_NAME portrait" `# Prompt dùng trong training` \
  --resolution=512 \
  --train_batch_size=1 `# Batch size = 1 hoặc 2 là an toàn cho Colab T4` \
  --gradient_accumulation_steps=2 `# Mô phỏng batch size lớn hơn (1*2=2)` \
  --learning_rate=1e-4 `# Learning rate` \
  --lr_scheduler="cosine" \
  --lr_warmup_steps=100 \
  --num_train_epochs=15 `# Số epochs (sẽ dùng cái này để dừng)` \
  # <<< DÒNG --max_train_steps=None ĐÃ BỊ XÓA >>> \
  --lora_rank=16 `# Rank của LoRA (thử 8, 16, 32)` \
  --seed=42 \
  --mixed_precision="fp16" `# Dùng float16` \
  --use_8bit_adam `# Dùng optimizer 8bit` \
  --gradient_checkpointing `# Tiết kiệm VRAM bằng gradient checkpointing` \
  --enable_xformers_memory_efficient_attention `# Dùng xformers` \
  --checkpointing_steps=500 `# Lưu checkpoint mỗi 500 steps` \
  --validation_prompt="$TRIGGER_WORD $CLASS_NAME girl, high quality" `# Prompt để tạo ảnh test trong lúc train` \
  --validation_epochs=2 `# Tạo ảnh test mỗi 2 epoch` \
  --report_to="tensorboard" # Báo cáo log cho tensorboard

echo "--- QUÁ TRÌNH HUẤN LUYỆN KẾT THÚC (nếu không có lỗi khác) ---" # echo thay vì print trong bash

--- QUÁ TRÌNH HUẤN LUYỆN KẾT THÚC (nếu không có lỗi khác) ---


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-04-28 01:46:19.319070: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745804779.339280    5269 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745804779.345516    5269 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-28 01:46:19.366652: I tensorflow/core/platform